<a href="https://colab.research.google.com/github/zrghassabi/VisionTransformers/blob/main/GANVisionTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Combining Generative Adversarial Networks (GANs) with Vision Transformers (ViTs) creates a powerful framework for various image generation tasks. GANs are used to generate realistic images, while ViTs can help enhance the quality of generated images or provide better representations for tasks like super-resolution or inpainting.

Here’s a high-level approach to combining GANs with Vision Transformers:

#Define the Vision Transformer Architecture:

This could be used in either the generator or discriminator, or both.

#Create the GAN Framework:
Set up the GAN components: the generator, discriminator, and the adversarial training process.

#Integrate the Vision Transformer: Incorporate the Vision Transformer into the GAN architecture.

#Example Code

Below is an example that integrates Vision Transformers into a GAN framework. In this example, the Vision Transformer is used in the discriminator to enhance its capability in image classification.

Vision Transformer Discriminator for GAN

# 1 Vision Transformer Block

In [ ]:
import torch
import torch.nn as nn

class VisionTransformerBlock(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_layers):
        super(VisionTransformerBlock, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, x):
        x = x.permute(1, 0, 2)  # [num_patches, batch_size, hidden_dim]
        x = self.transformer_encoder(x)
        return x.permute(1, 0, 2)  # [batch_size, num_patches, hidden_dim]


# 2 Discriminator with Vision Transformer

In [ ]:
class ViTDiscriminator(nn.Module):
    def __init__(self, img_size=64, patch_size=8, in_channels=3, hidden_dim=256, num_heads=4, num_layers=4):
        super(ViTDiscriminator, self).__init__()

        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_dim = patch_size * patch_size * in_channels

        # Patch Embedding Layer
        self.patch_embedding = nn.Linear(self.patch_dim, hidden_dim)

        # Positional Encoding
        self.position_embedding = nn.Parameter(torch.zeros(1, self.num_patches, hidden_dim))

        # Vision Transformer Block
        self.vit_block = VisionTransformerBlock(hidden_dim, num_heads, num_layers)

        # Classification Head
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        # Create patches
        batch_size = x.size(0)
        x = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size)
        x = x.contiguous().view(batch_size, self.patch_size * self.patch_size * x.size(1), -1)
        x = x.permute(0, 2, 1)  # [batch_size, num_patches, patch_dim]

        # Patch embedding
        x = self.patch_embedding(x)

        # Ensure positional encoding is properly sized
        if self.position_embedding.size(1) != x.size(1):
            self.position_embedding = nn.Parameter(torch.zeros(1, x.size(1), x.size(2)))

        # Add positional encoding
        x = x + self.position_embedding

        # Vision Transformer Block
        x = self.vit_block(x)

        # Use the output of the class token
        x = x.mean(dim=1)

        # Classification head
        x = self.fc(x)

        return x


# 3- GAN Framework

In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=3):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(z_dim, 128 * 8 * 8),
            nn.ReLU(True),
            nn.Unflatten(1, (128, 8, 8)),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

class GAN(nn.Module):
    def __init__(self, z_dim=100, img_size=64):
        super(GAN, self).__init__()
        self.generator = Generator(z_dim=z_dim)
        self.discriminator = ViTDiscriminator(img_size=img_size)

    def forward(self, z):
        return self.generator(z)

    def discriminate(self, img):
        return self.discriminator(img)


# 4 Training the GAN

In [ ]:
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# DataLoader Setup
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize images to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

# Initialize the GAN
z_dim = 100
gan = GAN(z_dim=z_dim, img_size=64)
criterion = nn.BCEWithLogitsLoss()
optimizer_g = optim.Adam(gan.generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = optim.Adam(gan.discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training Loop
num_epochs = 1
for epoch in range(num_epochs):
    for real_images, _ in train_loader:
        batch_size = real_images.size(0)
        real_labels = torch.ones(batch_size, 1)
        fake_labels = torch.zeros(batch_size, 1)

        # Train Discriminator
        optimizer_d.zero_grad()
        outputs = gan.discriminate(real_images)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        z = torch.randn(batch_size, z_dim)
        fake_images = gan(z)
        outputs = gan.discriminate(fake_images.detach())
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()
        optimizer_d.step()

        # Train Generator
        optimizer_g.zero_grad()
        outputs = gan.discriminate(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_g.step()

    print(f'Epoch {epoch+1}/{num_epochs}, D Loss: {d_loss_real.item() + d_loss_fake.item()}, G Loss: {g_loss.item()}')


Key Points:

#Vision Transformer Block:
Defines a transformer block used in the discriminator for learning image features.
#ViT Discriminator:
Uses Vision Transformer to classify real vs. fake images.

#Generator:
Simple fully connected network to generate images from random noise.

#Training:
Train the discriminator to differentiate between real and generated images, and train the generator to produce realistic images that can fool the discriminator.

This setup leverages the Vision Transformer for feature extraction and classification within a GAN framework, improving image quality and representation learning. You can adapt the architecture and training procedure to suit specific use cases or datasets.